In [1]:
import os
import glob
import re
import itertools

import pandas as pd
pd.options.display.max_columns = None
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [34]:
benchmarking_dir = '/cephfs/users/annawoodard/gene-fusion/FusionBenchmarking'
analysis_dir = 'simulated_data/sim_101'
ctat_dir = '/cephfs/users/annawoodard/gene-fusion/GRCh38_gencode_v31_CTAT_lib_Oct012019.plug-n-play/ctat_genome_lib_build_dir'

In [29]:
command = (
    "docker run -v {}:/data trinityctat/fusionbenchmarking "
    "bash -c "
    "'find /data/{}/samples -type f | /data/util/make_file_listing_input_table.pl'"
).format(benchmarking_dir, analysis_dir)

!$command > $benchmarking_dir/$analysis_dir/simulated_datafusion_result_file_listing.dat

In [24]:
command = (
    "docker run -v {}:/data trinityctat/fusionbenchmarking "
    "bash -c "
    "'/usr/local/src/FusionBenchmarking/benchmarking/collect_preds.pl "
    "/data/{}/simulated_datafusion_result_file_listing.dat'"
).format(benchmarking_dir, analysis_dir)
!$command > $benchmarking_dir/$analysis_dir/preds.collected

docker run -v /cephfs/users/annawoodard/gene-fusion/FusionBenchmarking:/data trinityctat/fusionbenchmarking bash -c '/usr/local/src/FusionBenchmarking/benchmarking/collect_preds.pl /data/simulated_data/sim_101/simulated_datafusion_result_file_listing.dat'


In [32]:
command = (
    "docker run -v {}:/data trinityctat/fusionbenchmarking "
    "bash -c '"
    "/usr/local/src/FusionBenchmarking/benchmarking/map_gene_symbols_to_gencode.pl "
    "/data/{}/preds.collected "
    "/data/resources/genes.coords.gz /data/resources/genes.aliases'"
).format(benchmarking_dir, analysis_dir)

!$command > $benchmarking_dir/$analysis_dir/preds.collected.gencode_mapped

docker run -v /cephfs/users/annawoodard/gene-fusion/FusionBenchmarking:/data trinityctat/fusionbenchmarking bash -c '/usr/local/src/FusionBenchmarking/benchmarking/map_gene_symbols_to_gencode.pl /data/simulated_data/sim_101/preds.collected /data/resources/genes.coords.gz /data/resources/genes.aliases'
-parsing annotation gene spans
-warning, no gene stored for identifier: [TRB@]
-warning, no gene stored for identifier: [TRG@]
-warning, no gene stored for identifier: [C8ORF44-SGK3]
-warning, no gene stored for identifier: [x]
-warning, no gene stored for identifier: [C1ORF234]
-warning, no gene stored for identifier: [C21ORF59-TCP10L]
-warning, no gene stored for identifier: [snoU13]
-warning, no gene stored for identifier: [C5ORF67]
-warning, no gene stored for identifier: [C17ORF112]
-warning, no gene stored for identifier: [C7ORF55-LUC7L2]


In [ ]:
!docker run -v /cephfs/users/annawoodard/gene-fusion/GRCh38_gencode_v31_CTAT_lib_Oct012019.plug-n-play/ctat_genome_lib_build_dir:/ctat_genome_lib_build_dir -v /cephfs/users/annawoodard/gene-fusion/data/interim/fusions.tsv:/fusions.tsv trinityctat/starfusion:1.8.0 /usr/local/src/STAR-Fusion/FusionInspector/FusionAnnotator/FusionAnnotator --annotate /fusions.tsv --full --genome_lib_dir /ctat_genome_lib_build_dir/ > /cephfs/users/annawoodard/gene-fusion/data/interim/annotated_fusions.tsv

In [35]:
command = (
    "docker run "
    "-v {}:/ctat_genome_lib_build_dir "
    "-v {}:/data "
    "trinityctat/fusionbenchmarking "
    "bash -c '"
    "FusionAnnotator/FusionAnnotator "
    "--annotate /data/{}/preds.collected.gencode_mapped -C 2 --genome_lib_dir /ctat_genome_lib_build_dir/'"
).format(ctat_dir, benchmarking_dir, analysis_dir)

!$command > $benchmarking_dir/$analysis_dir/preds.collected.gencode_mapped.wAnnot

In [36]:
command = (
    "docker run -v {}:/data trinityctat/fusionbenchmarking "
    "bash -c "
    "'/usr/local/src/FusionBenchmarking/benchmarking/filter_collected_preds.pl "
    "/data/{}/preds.collected.gencode_mapped.wAnnot'"
).format(benchmarking_dir, analysis_dir)
!$command > $benchmarking_dir/$analysis_dir/preds.collected.gencode_mapped.wAnnot.filt